<a href="https://colab.research.google.com/github/SasikiranJ/Handwritten-Digit-Recognition-by-building-ResNet50-From-Scratch-/blob/master/Handwritten%20digit%20recognition%20by%20building%20ResNet50%20from%20Scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#Importing essential libraries

import numpy as np

from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D,Add,Dropout
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from keras.utils.np_utils import to_categorical
from keras.initializers import glorot_uniform

import keras.backend as K
K.set_image_data_format('channels_last')
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img


%matplotlib inline

In [0]:
#Resnet Identity Block

def identity_block(X, f, filters, stage, block):
    
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    
    F1, F2, F3 = filters
    
    
    X_shortcut = X
    
    
    X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)
    
    
    
    
    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    
    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)
    X = Add()([X_shortcut, X])
    X = Activation('relu')(X)
    
    
    
    return X

In [0]:
# Convolution block

def convolutional_block(X, f, filters, stage, block, s = 2):
    
    
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value
    X_shortcut = X


    # MAIN PATH #
    # First component of main path 
    X = Conv2D(F1, (1, 1), strides = (s,s), name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)
    
    

    # Second component of main path 
    X = Conv2D(F2, (f, f), strides = (1,1),padding='same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path 
    X = Conv2D(F3, (1, 1), strides = (1,1), name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

    ##SHORTCUT PATH ## 
    X_shortcut = Conv2D(F3, (1, 1), strides = (s,s), name = conv_name_base + '1', kernel_initializer = glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3, name = bn_name_base + '1')(X_shortcut)

    # Add shortcut value to main path, and pass it through a RELU activation 
    X = Add()([X_shortcut,X])
    X = Activation('relu')(X)
    
    
    
    return X

In [0]:
#Resnet assembling with Five Stages

def ResNet50(input_shape = (28, 28, 1), classes = 10):

    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    
    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)
    
    # Stage 1
    X = Conv2D(64, (7, 7), strides = (2, 2), name = 'conv1', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = 'bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)
    

    # Stage 2
    X = convolutional_block(X, f = 3, filters = [64, 64, 256], stage = 2, block='a', s = 1)
    
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
    
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')
    

    

    # Stage 3 (≈4 lines)
    X = convolutional_block(X, f = 3, filters = [128, 128, 512], stage = 3, block='a', s = 2)
    
    X = identity_block(X, 3, [128,128, 512], stage=3, block='b')
    
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')
    

    # Stage 4 (≈6 lines)
    X = convolutional_block(X, f = 3, filters = [256, 256, 1024], stage = 4, block='a', s = 2)
    
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
    
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')
    

    # Stage 5 
    X = convolutional_block(X, f = 3, filters = [512, 512, 2048], stage = 5, block='a', s = 2)
    
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
    
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')
    

    # AVGPOOL
    X = AveragePooling2D(pool_size=(2, 2), strides=None, padding='same',name = 'avg_pool')(X)
    
    

    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0))(X)
    
    
    # Create model
    model = Model(inputs = X_input, outputs = X, name='ResNet50')

    return model

In [0]:

model = ResNet50(input_shape = (28, 28, 1), classes = 10)

In [98]:
#model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
top_model_weights_path = "best_model.h5"

# Data loading for testing resent50 (MNIST Dataset)
import tensorflow as tf
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
input_shape = (28, 28, 1)
# Making sure that the values are float so that we can get decimal points after division
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
# Normalizing the RGB codes by dividing it to the max RGB value.
x_train /= 255
x_test /= 255

#converting into categorical labels
from keras.utils import to_categorical
y_training = to_categorical(y_train)
y_testing = to_categorical(y_test)


print('x_train shape:', x_train.shape)
print('Number of images in x_train', x_train.shape[0])
print('Number of images in x_test', x_test.shape[0])
print('Number of images in x_test', y_test.shape[0])

x_train shape: (60000, 28, 28, 1)
Number of images in x_train 60000
Number of images in x_test 10000
Number of images in x_test (10000,)


In [0]:
from keras import optimizers
from keras.callbacks import EarlyStopping, ModelCheckpoint
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.adam(lr=1e-4),
              metrics=['acc'])  
# Defining callbacks to use early-stopping method for reducing computation after there is no improvement 
callbacks = [EarlyStopping(monitor='val_loss', patience=10,verbose=1,mode='auto'),
             ModelCheckpoint(filepath=top_model_weights_path, monitor='val_acc',verbose=1, save_best_only=True)]

#Fitting our model with training data and validation data
history = model.fit(x_train,y_training,
      batch_size=128,
      epochs=10,
      callbacks= callbacks,  
      validation_data=(x_test,y_testing)
)

In [120]:
#Loading optimal model file which gives good accuracy
from keras.models import load_model
model_opt = load_model("best_model.h5")

#Checking accuracy
(eval_loss, eval_accuracy) = model_opt.evaluate(  
 x_test, y_testing, batch_size=128, verbose=1)


print("[INFO] accuracy: {:.2f}%".format(eval_accuracy * 100))  
print("[INFO] Loss: {}".format(eval_loss))

10000/10000 [==============================] - 3s 298us/step
[INFO] accuracy: 98.58%
[INFO] Loss: 0.05827471232171811
